In [1]:
%cd /home/lcc/projects/MetaHarmonizer

/home/lcc/projects/MetaHarmonizer


/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import gevent.monkey

gevent.monkey.patch_all(thread=False, select=False)

True

In [3]:
import pandas as pd
from importlib import reload
from src.models import ontology_mapper_st as om_st
from src.Engine import ontology_mapping_engine as ome
import src.models.ontology_mapper_lm as om_lm
import src.models.ontology_mapper_rag_faiss as om_rag

reload(om_st)
reload(ome)
reload(om_lm)
reload(om_rag)

/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lcc/miniconda3/envs/py310/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lcc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lcc/nltk_data...
[nltk_data]   Package

<module 'src.models.ontology_mapper_rag_faiss' from '/home/lcc/projects/MetaHarmonizer/src/models/ontology_mapper_rag_faiss.py'>

In [4]:
from src.models.calc_stats import CalcStats

calc = CalcStats()

**Diseases**

In [5]:
df = pd.read_csv('./data/corpus/cbio_disease/disease_query_for_NCIT:C3262.csv')
large_corpus = pd.read_csv(
    'data/corpus/cbio_disease/disease_corpus_from_NCIT:C3262.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()
large_corpus_list = large_corpus['label'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

In [6]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [5]:
import os


os.environ["CUDA_VISIBLE_DEVICES"] = ""

LM Strategy

In [7]:
# LM Strategy w/ SapBERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='sap-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=large_corpus_list,
    cura_map=cura_map,
    om_strategy='lm',
    **other_params)
lm_sapbert_result = onto_engine_large.run()

25/06//2025 05:40:1750898425 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 05:40:1750898425 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 05:40:1750898425 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 05:40:1750898425 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
25/06//2025 05:40:1750898425 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module


No sentence-transformers model found with name model_cache/sap-bert. Creating a new one with mean pooling.
Batches: 100%|██████████| 7/7 [00:00<00:00, 75.88it/s]


In [8]:
lm_sapbert_accuracy_df = calc.calc_accuracy(lm_sapbert_result)
print(lm_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  68.319382
1  Top 3 Matches  76.947843
2  Top 5 Matches  79.974243


In [9]:
lm_sapbert_result.to_csv("lm_sapbert_result_disease.csv", index=False)

In [ ]:
# Language Model Strategy w/ PubMedBERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='pubmed-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=large_corpus_list,
    cura_map=cura_map,
    om_strategy='lm',
    **other_params)

lm_pubmedbert_result = onto_engine_large.run()

25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
25/06//2025 06:00:1750899659 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
25/06//2025 06:00:1750899659 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
25/06//2025 06:00:1750899659 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module


Batches: 100%|██████████| 4/4 [00:00<00:00, 58.51it/s]


In [12]:
# Language Model Strategy w/ PubMedBERT
lm_pubmedbert_accuracy_df = calc.calc_accuracy(lm_pubmedbert_result)
print(lm_pubmedbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  74.951706
1  Top 3 Matches  86.992917
2  Top 5 Matches  89.632968


In [13]:
lm_pubmedbert_result.to_csv("lm_pubmedbert_result.csv", index=False)

ST Strategy (PubMedBERT‑MS‑MARCO, SapBERT-mean-token)

In [7]:
# ST Strategy w/ SapBERT-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='mt-sap-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=large_corpus_list,
    cura_map=cura_map,
    om_strategy='st',
    **other_params)
st_sapbert_result = onto_engine_large.run()

24/06//2025 05:44:1750812292 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
24/06//2025 05:44:1750812292 PM - INFO - OntoMapEngine: Running Ontology Mapping
24/06//2025 05:44:1750812292 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
24/06//2025 05:44:1750812293 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
24/06//2025 05:44:1750812293 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module


No sentence-transformers model found with name model_cache/mt-sap-bert. Creating a new one with mean pooling.
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


In [9]:
st_sapbert_result.to_csv("st_sapbert_result.csv", index=False)

In [8]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  69.478429
1  Top 3 Matches  77.656149
2  Top 5 Matches  80.296201


In [10]:
# st - pubmed-bert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='pubmed-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=large_corpus_list,
    cura_map=cura_map,
    om_strategy='st',
    **other_params)

st_pubmetbert_result = onto_engine_large.run()

24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Running Ontology Mapping
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Running Ontology Mapping
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
24/06//2025 05:48:1750812503 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
24/06//2025 05:48:1750812503 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module
24/06//2025 05:48:1750812503 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module


Batches: 100%|██████████| 7/7 [00:00<00:00, 77.47it/s]
Exception in worker
Traceback (most recent call last):
  File "/home/lcc/miniconda3/envs/py310/lib/python3.10/concurrent/futures/thread.py", line 81, in _worker
    work_item = work_queue.get(block=True)
  File "src/gevent/queue.py", line 381, in gevent._gevent_cqueue.SimpleQueue.get
  File "src/gevent/queue.py", line 397, in gevent._gevent_cqueue.SimpleQueue.get
  File "src/gevent/queue.py", line 373, in gevent._gevent_cqueue.SimpleQueue._SimpleQueue__get_or_peek
  File "src/gevent/_waiter.py", line 154, in gevent._gevent_c_waiter.Waiter.get
  File "src/gevent/_greenlet_primitives.py", line 65, in gevent._gevent_c_greenlet_primitives.SwitchOutGreenletWithLoop.switch
  File "src/gevent/_gevent_c_greenlet_primitives.pxd", line 35, in gevent._gevent_c_greenlet_primitives._greenlet_switch
gevent.exceptions.LoopExit: This operation would block forever
	Hub: <Hub '' at 0x7fc500b8d670 epoll pending=0 ref=0 fileno=166 thread_ident=0x7fc4e

In [11]:
# Sentences Transformer Strategy w/ PubMedBERT
st_pubmedbert_accuracy_df = calc.calc_accuracy(st_pubmetbert_result)
print(st_pubmedbert_accuracy_df)

  Accuracy Level   Accuracy
0    Top 1 Match  65.679330
1  Top 3 Matches  73.792659
2  Top 5 Matches  76.819060


In [12]:
st_pubmetbert_result.to_csv("st_pubmedbert_result.csv", index=False)

RAG

In [14]:
# RAG Strategy w/ SapBERT
rag_engine = ome.OntoMapEngine(method='mt-sap-bert',
                               category='disease',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               om_strategy='rag',
                               test_or_prod='test')
rag_sapbert_result = rag_engine.run()

25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Running Ontology Mapping
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
25/06//2025 09:03:1750910609 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
25

AttributeError: can't set attribute 'vector_store'

In [ ]:
rag_sapbert_result.to_csv("rag_sapbert_result_disease.csv",
                  index=False)

In [ ]:
# RAG Strategy w/ SapBERT
rag_sapbert_accuracy_df = calc.calc_accuracy(rag_sapbert_result)
print(rag_sapbert_accuracy_df)

In [ ]:
# RAG Strategy w/ pubmed-bert
rag_engine = ome.OntoMapEngine(method='pubmed-bert',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               yaml_path='./src/models/method_model.yaml',
                               om_strategy='rag',
                               test_or_prod='test')
rag_pubmed_result = rag_engine.run()

In [ ]:
# RAG Strategy w/ pubmed-bert
rag_pubmed_accuracy_df = calc.calc_accuracy(rag_pubmed_result)
print(rag_pubmed_accuracy_df)

In [ ]:
rag_pubmed_result.to_csv("rag_pubmed_result_disease.csv",
                  index=False)

In [ ]:
rag_pubmed_result

Overlapping of different strategies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


def compute_overlap_matrix(data_dict):
    methods = list(data_dict.keys())
    matrix = pd.DataFrame(index=methods, columns=methods)

    for m1 in methods:
        for m2 in methods:
            if m1 == m2:
                matrix.loc[m1, m2] = 1.0
            else:
                intersect = len(data_dict[m1] & data_dict[m2])
                union = len(data_dict[m1] | data_dict[m2])
                matrix.loc[m1, m2] = round(intersect / union, 3)
    return matrix


def plot_heatmap(matrix, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(matrix.astype(float),
                annot=True,
                fmt=".2f",
                cmap="Blues",
                linewidths=0.5)
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
files = {
    'sapbert_lm':
    'lm_sapbert_result.csv',
    'sapbert_st':
    'st_sapbert_result.csv',
    'sapbert_rag':
    'rag_sapbert_result.csv',
    'pubmed_lm':
    'lm_pubmedbert_result.csv',
    'pubmed_st':
    'st_pubmedbert_result.csv',
    'pubmed_rag':
    'rag_pubmed_result_disease.csv',
}

top1_dict = {}
top5_dict = {}

for name, path in files.items():
    df = pd.read_csv(path)
    top1_dict[name] = set(df[df['match_level'] == 1]['original_value'])
    top5_dict[name] = set(df[df['match_level'] <= 5]['original_value'])

top1_overlap = compute_overlap_matrix(top1_dict)
top5_overlap = compute_overlap_matrix(top5_dict)

print("🔹 Top1 Jaccard Overlap Matrix")
print(top1_overlap)

print("\n🔹 Top5 Jaccard Overlap Matrix")
print(top5_overlap)

In [ ]:
plot_heatmap(top1_overlap, "Disease Top-1 Jaccard Overlap")
plot_heatmap(top5_overlap, "Disease Top-5 Jaccard Overlap")

Sample

In [ ]:
import sqlite3

conn = sqlite3.connect('src/KnowledgeDb/vector_db.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT MAX(LENGTH(context)) FROM mt_sap_bert_term_info;")
max_len = cursor.fetchone()[0]

print("Max length of 'context' column:", max_len)

conn.close()

In [ ]:
import sqlite3
import csv

conn = sqlite3.connect('src/KnowledgeDb/version1/vector_db.sqlite')
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM mt_sap_bert_term_info
    WHERE LENGTH(context) > 2200;
""")

rows = cursor.fetchall()

print(
    f"✅ Exported {len(rows)} rows with long contexts."
)

conn.close()


**Treatment**

In [ ]:
# Updated code to pass the dict rather than a DataFrame
df = pd.read_csv(
    './data/corpus/cbio_treatment_name/trt_name_query_for_NCIT:C1909.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

LM Strategy (SapBERT)

In [ ]:
# LM Strategy w/ SapBERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='sap-bert',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    yaml_path='./src/models/method_model.yaml',
    om_strategy='lm',
    **other_params)
lm_sapbert_result = onto_engine_large.run()

In [ ]:
lm_sapbert_accuracy_df = calc.calc_accuracy(lm_sapbert_result)
print(lm_sapbert_accuracy_df)

In [ ]:
lm_sapbert_result.to_csv("lm_sapbert_result_trt.csv", index=False)

In [ ]:
# Language Model Strategy w/ PubMedBERT
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='pubmed-bert',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    yaml_path='./src/models/method_model.yaml',
    om_strategy='lm',
    **other_params)

lm_pubmedbert_result = onto_engine_large.run()

In [ ]:
# Language Model Strategy w/ PubMedBERT
lm_pubmedbert_accuracy_df = calc.calc_accuracy(lm_pubmedbert_result)
print(lm_pubmedbert_accuracy_df)

In [ ]:
lm_pubmedbert_result.to_csv("lm_pubmedbert_result_trt.csv", index=False)

ST Strategy (PubMedBERT‑MS‑MARCO, SapBERT-mean-token)

In [ ]:
# ST Strategy w/ SapBERT-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='mt-sap-bert',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    yaml_path='./src/models/method_model.yaml',
    om_strategy='st',
    **other_params)
st_sapbert_result = onto_engine_large.run()

In [ ]:
st_sapbert_result.to_csv("st_sapbert_result_trt.csv", index=False)

In [ ]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

In [ ]:
# st - pubmed-bert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='pubmed-bert',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    yaml_path='./src/models/method_model.yaml',
    om_strategy='st',
    **other_params)

st_pubmetbert_result = onto_engine_large.run()

In [ ]:
# Sentences Transformer Strategy w/ PubMedBERT
st_pubmedbert_accuracy_df = calc.calc_accuracy(st_pubmetbert_result)
print(st_pubmedbert_accuracy_df)

In [ ]:
st_pubmetbert_result.to_csv("st_pubmedbert_result_trt.csv", index=False)

In [ ]:
for df, name in [
    (lm_sapbert_accuracy_df, ("SapBERT", "LM")),
    (lm_pubmedbert_accuracy_df, ("PubMedBERT", "LM")),
    (st_sapbert_accuracy_df, ("SapBERT", "ST")),
    (st_pubmedbert_accuracy_df, ("PubMedBERT", "ST")),
]:
    df["Model"] = name[0]
    df["Type"] = name[1]  # LM or ST

all_df = pd.concat([
    lm_sapbert_accuracy_df,
    lm_pubmedbert_accuracy_df,
    st_sapbert_accuracy_df,
    st_pubmedbert_accuracy_df
])

plt.figure(figsize=(8, 5))

model_palette = {
    "SapBERT": "#1f77b4",      
    "PubMedBERT": "#ff7f0e"    
}
linestyle_map = {
    "LM": "-",
    "ST": "--"
}

for (model, typ), subdf in all_df.groupby(["Model", "Type"]):
    plt.plot(
        subdf["Accuracy Level"],
        subdf["Accuracy"],
        label=f"{typ} + {model}",
        linestyle=linestyle_map[typ],
        color=model_palette[model],
        marker="o"
    )

plt.title("Top-K Accuracy Comparison")
plt.ylabel("Accuracy (%)")
plt.xlabel("Top-K Level")
plt.ylim(65, 95)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(title="Model Combination")
plt.tight_layout()
plt.show()

RAG

In [ ]:
# RAG Strategy w/ SapBERT
rag_engine = ome.OntoMapEngine(method='mt-sap-bert',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               yaml_path='./src/models/method_model.yaml',
                               om_strategy='rag',
                               test_or_prod='test')
rag_sapbert_result = rag_engine.run()

In [ ]:
rag_sapbert_result.to_csv("rag_sapbert_result_trt.csv",
                  index=False)

In [ ]:
# RAG Strategy w/ SapBERT
rag_sapbert_accuracy_df = calc.calc_accuracy(rag_sapbert_result)
print(rag_sapbert_accuracy_df)

In [ ]:
# RAG Strategy w/ pubmed-bert
rag_engine = ome.OntoMapEngine(method='pubmed-bert',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               yaml_path='./src/models/method_model.yaml',
                               om_strategy='rag',
                               test_or_prod='test')
rag_pubmed_result = rag_engine.run()

In [ ]:
rag_pubmed_result.to_csv("rag_pubmed_result_trt.csv",
                         index=False)

In [ ]:
# RAG Strategy w/ pubmed-bert
rag_pubmed_accuracy_df = calc.calc_accuracy(rag_pubmed_result)
print(rag_pubmed_accuracy_df)

Comparison

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = r"data/outputs/Comparison.xlsx"
df = pd.read_excel(file_path)

df["model"] = df["model"].str.replace("–", "-", regex=False)  # en dash
df["model"] = df["model"].str.replace("‑", "-",
                                      regex=False)  # non-breaking hyphen

categories = df["category"].unique()
topks = ["Top 1", "Top 3", "Top 5"]
colors = {"bodysite": "blue", "disease": "orange", "treatment": "green"}
linestyles = {"SapBERT": "solid", "PubMedBERT-MS-MARCO": "dashed"}

plt.figure(figsize=(8, 6))

for cat in categories:
    df_sapbert = df[(df["category"] == cat) & (df["model"] == "SapBERT")]
    if not df_sapbert.empty:
        mean_accuracy = df_sapbert[topks].mean()
        label = f"{cat.capitalize()} - SapBERT"
        plt.plot(topks,
                 mean_accuracy,
                 label=label,
                 color=colors[cat],
                 linestyle=linestyles["SapBERT"])

    df_pubmed_mean = df[(df["category"] == cat)
                        & (df["model"] == "PubMedBERT-MS-MARCO") &
                        (df["pooling strategy"] == "lm")]
    if not df_pubmed_mean.empty:
        mean_accuracy = df_pubmed_mean[topks].mean()
        label = f"{cat.capitalize()} - PubMedBERT"
        plt.plot(topks,
                 mean_accuracy,
                 label=label,
                 color=colors[cat],
                 linestyle=linestyles["PubMedBERT-MS-MARCO"])

plt.title("Accuracy Comparison between SapBERT and PubMedBERT")
plt.xlabel("Top K Match")
plt.ylabel("Accuracy (%)")
plt.ylim(60, 100)
plt.grid(True)
plt.legend(title="Model")
plt.tight_layout()
plt.show()

In [ ]:
!curl "https://api-evsrest.nci.nih.gov/api/v1/metadata/ncit/roles"



In [ ]:
import requests
import pandas as pd

url = "https://api-evsrest.nci.nih.gov/api/v1/metadata/ncit/roles"
response = requests.get(url)
roles = response.json()

df = pd.DataFrame(roles)
df.to_csv("ncit_roles.csv", index=False)
print(df.head())

Bodysite

In [ ]:
df = pd.read_csv(
    './data/corpus/cbio_body_site/body_site_query_for_NCIT:C32221.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

In [ ]:
# RAG Strategy w/ sap-bert
rag_engine = ome.OntoMapEngine(method='mt-sap-bert',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               yaml_path='./src/models/method_model.yaml',
                               om_strategy='rag',
                               test_or_prod='test')
rag_sapbert_result = rag_engine.run()

In [ ]:
rag_sapbert_result.to_csv("rag_sapbert_result_bodysite.csv",
                  index=False)

In [ ]:
rag_sapbert_accuracy_df = calc.calc_accuracy(rag_sapbert_result)
print(rag_sapbert_accuracy_df)

In [ ]:
# RAG Strategy w/ pubmed-bert
rag_engine = ome.OntoMapEngine(method='pubmed-bert',
                               topk=5,
                               query=query_list,
                               corpus=small_corpus_list,
                               cura_map=cura_map,
                               yaml_path='./src/models/method_model.yaml',
                               om_strategy='rag',
                               test_or_prod='test')
rag_pubmed_result = rag_engine.run()

In [ ]:
rag_pubmed_result.to_csv("rag_pubmed_result_bodysite.csv",
                  index=False)

In [ ]:
rag_pubmed_accuracy_df = calc.calc_accuracy(rag_pubmed_result)
print(rag_pubmed_accuracy_df)

Overlapping of different strategies

In [ ]:
files = {
    'sapbert_lm':
    'lm_sapbert_result_trt.csv',
    'sapbert_st':
    'st_sapbert_result_trt.csv',
    'sapbert_rag':
    'rag_sapbert_result_trt.csv',
    'pubmed_lm':
    'lm_pubmedbert_result_trt.csv',
    'pubmed_st':
    'st_pubmedbert_result_trt.csv',
    'pubmed_rag':
    'rag_pubmed_result_trt.csv',
}

top1_dict = {}
top5_dict = {}

for name, path in files.items():
    df = pd.read_csv(path)
    top1_dict[name] = set(df[df['match_level'] == 1]['original_value'])
    top5_dict[name] = set(df[df['match_level'] <= 5]['original_value'])

top1_overlap = compute_overlap_matrix(top1_dict)
top5_overlap = compute_overlap_matrix(top5_dict)

print("🔹 Top1 Jaccard Overlap Matrix")
print(top1_overlap)

print("\n🔹 Top5 Jaccard Overlap Matrix")
print(top5_overlap)

In [ ]:
plot_heatmap(top1_overlap, "Treatment Top-1 Jaccard Overlap")
plot_heatmap(top5_overlap, "Treatment Top-5 Jaccard Overlap")

Test Sample

Find token count of a term

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
termList = [
    "Transformed B-Cell Lymphoma, Unclassifiable, with Features Intermediate between Diffuse Large B-Cell Lymphoma and Classic Hodgkin Lymphoma to Diffuse Large B-Cell Lymphoma",
    "Stage IV Childhood Malignant Kidney Neoplasm Excluding Renal Cell Carcinoma by Toronto Guidelines v2, International Society of Pediatric Oncology (SIOP) Protocol",
    "Localized Childhood Malignant Kidney Neoplasm Excluding Renal Cell Carcinoma by Toronto Guidelines v2, International Society of Pediatric Oncology (SIOP) Protocol",
    "Diphtheria Toxoid/Tetanus Toxoid/Acellular Pertussis Adsorbed, Recombinant Hepatitis B/Inactivated Poliovirus Vaccine Combined",
    "Stage IVC Unknown Primary Tumor (Except for EBV-Related and HPV-Related Tumors) and Metastatic Cervical Adenopathy AJCC v8",
    "Chrysanthemum morifolium/Ganoderma lucidum/Glycyrrhiza glabra/Isatis indigotica/Panax pseudoginseng/Rabdosia rubescens/Scutellaria baicalensis/Serona repens Supplement"
]

for term in termList:
    tokens = tokenizer(term, return_tensors='pt')
    token_count = len(tokens['input_ids'][0])

    print(f"Input term: {term}")
    print(f"Token count: {token_count}")
    # print(f"Tokens: {tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])}")

Input term: Transformed B-Cell Lymphoma, Unclassifiable, with Features Intermediate between Diffuse Large B-Cell Lymphoma and Classic Hodgkin Lymphoma to Diffuse Large B-Cell Lymphoma
Token count: 32
Input term: Stage IV Childhood Malignant Kidney Neoplasm Excluding Renal Cell Carcinoma by Toronto Guidelines v2, International Society of Pediatric Oncology (SIOP) Protocol
Token count: 27
Input term: Localized Childhood Malignant Kidney Neoplasm Excluding Renal Cell Carcinoma by Toronto Guidelines v2, International Society of Pediatric Oncology (SIOP) Protocol
Token count: 26
Input term: Diphtheria Toxoid/Tetanus Toxoid/Acellular Pertussis Adsorbed, Recombinant Hepatitis B/Inactivated Poliovirus Vaccine Combined
Token count: 25
Input term: Stage IVC Unknown Primary Tumor (Except for EBV-Related and HPV-Related Tumors) and Metastatic Cervical Adenopathy AJCC v8
Token count: 29
Input term: Chrysanthemum morifolium/Ganoderma lucidum/Glycyrrhiza glabra/Isatis indigotica/Panax pseudoginseng/R

Small corpora

In [5]:
df = pd.read_csv('random_20_samples.csv')

query_list = df['original_value'].tolist()
small_corpus_list = df['curated_ontology'].tolist()

cura_map = dict(zip(df['original_value'], df['curated_ontology']))

In [8]:
# Test network connectivity
import requests
try:
    response = requests.get('https://huggingface.co', timeout=10, verify=False)
    print(f"Network connection is working: {response.status_code}")
except Exception as e:
    print(f"Network connection failed: {e}")

Network connection failed: maximum recursion depth exceeded while calling a Python object


In [7]:
# ST Strategy w/ SapBERT-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(
    method='mt-sap-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    om_strategy='st',
    **other_params)
st_sapbert_result = onto_engine_large.run()

24/06//2025 01:56:1750755376 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
24/06//2025 01:56:1750755376 AM - INFO - OntoMapEngine: Running Ontology Mapping
24/06//2025 01:56:1750755376 AM - INFO - OntoMapEngine: Separating exact and non-exact matches
24/06//2025 01:56:1750755376 AM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
24/06//2025 01:56:1750755376 AM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module
24/06//2025 01:56:1750755376 AM - INFO - OntoMapST.get_match_results: Creating embeddings for query_list and corpus


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext-mean-token. Creating a new one with mean pooling.


24/06//2025 01:56:1750755381 AM - INFO - OntoMapST.get_match_results: Calculating cosine similarity matrix
24/06//2025 01:56:1750755381 AM - INFO - OntoMapST.get_match_results: Generating results table
24/06//2025 01:56:1750755381 AM - INFO - OntoMapST.get_match_results: Results Generated


In [8]:
st_sapbert_accuracy_df = calc.calc_accuracy(st_sapbert_result)
print(st_sapbert_accuracy_df)

  Accuracy Level  Accuracy
0    Top 1 Match      95.0
1  Top 3 Matches     100.0
2  Top 5 Matches     100.0


In [8]:
st_sapbert_result

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,top1_accuracy,top3_accuracy,top5_accuracy
0,ACUTE MEGAKARYOBLASTIC LEUKAEMIA,Acute Megakaryoblastic Leukemia,1,2,Acute Megakaryoblastic Leukemia,0.9901,Thyroid Gland Anaplastic Carcinoma,0.4113,Hilar Cholangiocarcinoma,0.3963,Thyroid Gland Sarcoma,0.3907,Small Cell Osteosarcoma,0.3826,True,True,True
1,ANAPLASTIC THYROID CANCER,Thyroid Gland Anaplastic Carcinoma,1,2,Thyroid Gland Anaplastic Carcinoma,0.9535,Thyroid Gland Carcinoma,0.7319,Papillary Carcinoma,0.6344,Thyroid Gland Sarcoma,0.6341,Hilar Cholangiocarcinoma,0.5788,True,True,True
2,"CARCINOMA, PAPILLARY",Papillary Carcinoma,1,2,Papillary Carcinoma,0.9636,Thyroid Gland Carcinoma,0.6779,Breast Papillary Ductal Carcinoma with Invasion,0.6327,Thyroid Gland Anaplastic Carcinoma,0.6072,Hilar Cholangiocarcinoma,0.4950,True,True,True
3,CERVICAL ENDOMETRIOID CARCINOMA,Cervical Endometrioid Adenocarcinoma,1,2,Cervical Endometrioid Adenocarcinoma,0.9611,Chondrosarcoma,0.4979,Ovarian High Grade Serous Adenocarcinoma,0.4873,Papillary Carcinoma,0.4603,Thyroid Gland Carcinoma,0.4477,True,True,True
4,CHONDROSARCOMA,Chondrosarcoma,1,2,Chondrosarcoma,1.0000,Small Cell Osteosarcoma,0.5933,Thyroid Gland Sarcoma,0.5162,Sacrococcygeal Teratoma,0.4845,Cervical Endometrioid Adenocarcinoma,0.4698,True,True,True
5,CLEAR CELL BORDERLINE OVARIAN TUMOR,Borderline Ovarian Clear Cell Tumor,1,2,Borderline Ovarian Clear Cell Tumor,0.9814,Borderline Ovarian Serous Tumor-Micropapillary...,0.6298,Ovarian High Grade Serous Adenocarcinoma,0.5472,Small Cell Osteosarcoma,0.5026,Papillary Carcinoma,0.4581,True,True,True
6,GASTROINTESTINAL NEUROENDOCRINE,Digestive System Neuroendocrine Tumor,1,2,Digestive System Neuroendocrine Tumor,0.8244,Neuroendocrine Neoplasm,0.7846,Primitive Neuroectodermal Tumor,0.5252,Papillary Carcinoma,0.4477,Thyroid Gland Sarcoma,0.4303,True,True,True
7,HEAD & NECK SQUAMOUS CELL CARCINOMA BASALOID TYPE,Head and Neck Basaloid Carcinoma,1,2,Head and Neck Basaloid Carcinoma,0.8953,Chondrosarcoma,0.3669,Cervical Endometrioid Adenocarcinoma,0.3444,Small Cell Osteosarcoma,0.3368,Ovarian High Grade Serous Adenocarcinoma,0.3328,True,True,True
8,HGS,Ovarian High Grade Serous Adenocarcinoma,1,2,Ovarian High Grade Serous Adenocarcinoma,0.6497,Small Cell Osteosarcoma,0.5155,Thyroid Gland Sarcoma,0.4975,Non-Hodgkin Lymphoma,0.4520,Borderline Ovarian Clear Cell Tumor,0.4440,True,True,True
9,MALIGNANT SACROCOCCYGEAL TERATOMA,Sacrococcygeal Teratoma,1,2,Sacrococcygeal Teratoma,0.9282,Chondrosarcoma,0.4668,Thyroid Gland Sarcoma,0.3897,Cervical Endometrioid Adenocarcinoma,0.3852,Ovarian High Grade Serous Adenocarcinoma,0.3825,True,True,True


In [6]:
# rag Strategy w/ SapBERT-mean-token
other_params = {"test_or_prod": "test"}
onto_engine_small = ome.OntoMapEngine(
    method='mt-sap-bert',
    category='disease',
    topk=5,
    query=query_list,
    corpus=small_corpus_list,
    cura_map=cura_map,
    om_strategy='rag',
    **other_params)
rag_sapbert_result = onto_engine_small.run()

24/06//2025 02:19:1750756777 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
24/06//2025 02:19:1750756777 AM - INFO - OntoMapEngine: Running Ontology Mapping
24/06//2025 02:19:1750756777 AM - INFO - OntoMapEngine: Separating exact and non-exact matches
24/06//2025 02:19:1750756777 AM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
24/06//2025 02:19:1750756777 AM - INFO - OntoMapRAG: Initialized OntoMapRAG module


No sentence-transformers model found with name model_cache/mt-sap-bert. Creating a new one with mean pooling.


Error fetching NCI code for 'Borderline Ovarian Serous Tumor-Micropapillary Variant': No NCI code found for term 'Borderline Ovarian Serous Tumor-Micropapillary Variant'
Error fetching NCI code for 'Breast Papillary Ductal Carcinoma with Invasion': No NCI code found for term 'Breast Papillary Ductal Carcinoma with Invasion'
24/06//2025 02:19:1750756799 AM - INFO - OntoMapRAG: Generating results table


Processing queries: 100%|██████████| 20/20 [00:00<00:00, 73.59it/s]

24/06//2025 02:20:1750756800 AM - INFO - OntoMapRAG: Results Generated


In [7]:
rag_sapbert_accuracy_df = calc.calc_accuracy(rag_sapbert_result)
print(rag_sapbert_accuracy_df)

  Accuracy Level  Accuracy
0    Top 1 Match      85.0
1  Top 3 Matches      90.0
2  Top 5 Matches      90.0


In [8]:
rag_sapbert_result

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score,top1_accuracy,top3_accuracy,top5_accuracy
0,ACUTE MEGAKARYOBLASTIC LEUKAEMIA,Acute Megakaryoblastic Leukemia,1,2,Acute Megakaryoblastic Leukemia,0.5078,Non-Hodgkin Lymphoma,0.2428,Borderline Ovarian Clear Cell Tumor,0.2339,Small Cell Osteosarcoma,0.2223,Chondrosarcoma,0.2209,True,True,True
1,ANAPLASTIC THYROID CANCER,Thyroid Gland Anaplastic Carcinoma,2,2,Thyroid Gland Carcinoma,0.4504,Thyroid Gland Anaplastic Carcinoma,0.4276,Thyroid Gland Sarcoma,0.3611,Chondrosarcoma,0.3332,Small Cell Osteosarcoma,0.2789,False,True,True
2,"CARCINOMA, PAPILLARY",Papillary Carcinoma,1,2,Papillary Carcinoma,0.4744,Thyroid Gland Carcinoma,0.3534,Chondrosarcoma,0.2842,Neuroendocrine Neoplasm,0.2718,Hilar Cholangiocarcinoma,0.2701,True,True,True
3,CERVICAL ENDOMETRIOID CARCINOMA,Cervical Endometrioid Adenocarcinoma,1,2,Cervical Endometrioid Adenocarcinoma,0.4088,Chondrosarcoma,0.3142,Ovarian High Grade Serous Adenocarcinoma,0.2395,Borderline Ovarian Clear Cell Tumor,0.2342,Small Cell Osteosarcoma,0.2215,True,True,True
4,CHONDROSARCOMA,Chondrosarcoma,1,2,Chondrosarcoma,0.5808,Small Cell Osteosarcoma,0.3781,Sacrococcygeal Teratoma,0.3235,Head and Neck Basaloid Carcinoma,0.2869,Neuroendocrine Neoplasm,0.2641,True,True,True
5,CLEAR CELL BORDERLINE OVARIAN TUMOR,Borderline Ovarian Clear Cell Tumor,1,2,Borderline Ovarian Clear Cell Tumor,0.6355,Ovarian High Grade Serous Adenocarcinoma,0.4191,Small Cell Osteosarcoma,0.3822,Chondrosarcoma,0.3636,Sacrococcygeal Teratoma,0.3526,True,True,True
6,GASTROINTESTINAL NEUROENDOCRINE,Digestive System Neuroendocrine Tumor,1,2,Digestive System Neuroendocrine Tumor,0.5703,Neuroendocrine Neoplasm,0.5199,Primitive Neuroectodermal Tumor,0.3923,Borderline Ovarian Clear Cell Tumor,0.3355,Head and Neck Basaloid Carcinoma,0.3241,True,True,True
7,HEAD & NECK SQUAMOUS CELL CARCINOMA BASALOID TYPE,Head and Neck Basaloid Carcinoma,1,2,Head and Neck Basaloid Carcinoma,0.4656,Chondrosarcoma,0.2417,Thyroid Gland Carcinoma,0.2382,Thyroid Gland Anaplastic Carcinoma,0.2304,Neuroendocrine Neoplasm,0.2226,True,True,True
8,HGS,Ovarian High Grade Serous Adenocarcinoma,1,2,Ovarian High Grade Serous Adenocarcinoma,0.3557,Borderline Ovarian Clear Cell Tumor,0.3142,Non-Hodgkin Lymphoma,0.2763,Small Cell Osteosarcoma,0.2689,Chondrosarcoma,0.2673,True,True,True
9,MALIGNANT SACROCOCCYGEAL TERATOMA,Sacrococcygeal Teratoma,1,2,Sacrococcygeal Teratoma,0.5818,Chondrosarcoma,0.3544,Borderline Ovarian Clear Cell Tumor,0.2177,Cervical Endometrioid Adenocarcinoma,0.2098,Ovarian High Grade Serous Adenocarcinoma,0.2097,True,True,True
